# Protected Area Boundary Change

This notebook outlines the methodology used to measure at protected area boundaries via GEE. 

The notebook queries MODIS imagery and returns the gradient values of relevant bands as well as the vegetation indices NDVI and EVI. This code expects an annual time span and returns a geoTIFF for each band in each park for that year.

In [ ]:
import ee
import geemap
import numpy as np
import pandas as pd

In [ ]:
service_account_key = 'service_account_key.json'
credentials = ee.ServiceAccountCredentials('avanscoyoc@berkeley.edu', service_account_key)
ee.Initialize(credentials)

print(ee.String('Hello from the Earth Engine servers!').getInfo())

### Load dataset

In [ ]:
# Load protected areas dataset
protected_areas = ee.FeatureCollection("WCMC/WDPA/202106/polygons")

# Define filters
marine_filter = ee.Filter.eq("MARINE", "0")
not_mpa_filter = ee.Filter.neq("DESIG_ENG", "Marine Protected Area")
status_filter = ee.Filter.inList("STATUS", ["Designated", "Established", "Inscribed"])
designation_filter = ee.Filter.neq("DESIG_ENG", "UNESCO-MAB Biosphere Reserve")
excluded_pids = ["555655917", "555656005", "555656013", "555665477", "555656021",
                 "555665485", "555556142", "187", "555703455", "555563456", "15894"]
area_filter = ee.Filter.gte("GIS_AREA", 200)

# Filter to exclude specific WDPA_PID values
pids_filter = ee.Filter.inList("WDPA_PID", excluded_pids).Not()

# Combine all filters
combined_filter = ee.Filter.And(
    marine_filter,
    not_mpa_filter,
    status_filter,
    designation_filter,
    pids_filter,
    area_filter
)

# Apply filters
data = protected_areas.filter(combined_filter)

# Get WDPA_PIDs
wdpa_pids = data.aggregate_array("WDPA_PID").getInfo()
print(f"Number of protected areas: {len(wdpa_pids)}")

### Set global constants

In [ ]:
PROTECTED_AREAS = data
WATER_MASK = ee.Image('JRC/GSW1_0/GlobalSurfaceWater')
MODIS = ee.ImageCollection('MODIS/006/MOD09A1')
BUFFER = 10000
MAX_ERR = 1
YEAR = 2010

### Define functions

In [ ]:
def buffer_polygon(feat, buff, err):
    """Create buffer around polygon"""
    feat = ee.Feature(feat)
    out = feat.buffer(buff).geometry()  # 10km out
    inn = feat.buffer(-buff).geometry()  # 10km in
    aoi = out.difference(inn, err)
    return aoi

def mask_water(feat):
    """Mask water bodies from feature"""
    water_no_holes = WATER_MASK.select('max_extent')\
        .focalMax(radius=30, units='meters', kernelType='square')\
        .focalMin(radius=30, units='meters', kernelType='square')
    water_vect = water_no_holes.reduceToVectors(
        reducer=ee.Reducer.countEvery(),
        geometry=feat.buffer(1000),
        scale=30,
        maxPixels=1e10,
        geometryType='polygon',
        eightConnected=False)
    geom = feat.difference(water_vect.geometry(), maxError=1)
    return geom

def filter_for_year(feat, year):
    """Filter images for specific year"""
    start = ee.Date.fromYMD(year, 1, 1)
    return ee.Filter.And(
        ee.Filter.bounds(feat),
        ee.Filter.date(start, start.advance(1, "year"))
    )

def add_indices_to_image(image):
    """Add vegetation indices to image"""
    EVI = image.expression(
        "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))",
        {
            'NIR': image.select("sur_refl_b02"),
            'RED': image.select("sur_refl_b01"),
            'BLUE': image.select("sur_refl_b03")
        }
    ).rename("EVI")
    
    NDVI = image.expression(
        "(NIR - RED) / (NIR + RED)",
        {
            'NIR': image.select("sur_refl_b02"),
            'RED': image.select("sur_refl_b01")
        }
    ).rename("NDVI")
    
    return image.addBands([EVI, NDVI])

def get_gradient_magnitude(image):
    """Calculate gradient magnitude"""
    gradient = image.gradient()
    gradient_x = gradient.select('x')
    gradient_y = gradient.select('y')
    magnitude = gradient_x.pow(2).add(gradient_y.pow(2)).sqrt()
    return magnitude

In [ ]:
# Define test geometry
geometry = ee.Geometry.Polygon([
    [
        [-119.80738088038615, 38.53793770547406],
        [-120.35120412257365, 38.64956590608609],
        [-120.86206838038615, 38.51215270703278],
        [-120.41162892726115, 38.05942162797243],
        [-120.13697072413615, 38.24085287086409],
        [-120.05457326319865, 37.98585693208372],
        [-119.85681935694865, 38.01182937540273],
        [-119.79639455226115, 38.11130517090483],
        [-119.71399709132365, 38.119948848017394],
        [-119.63709279444865, 38.1674707827119],
        [-119.62061330226115, 38.2494811940204],
        [-119.65357228663615, 38.32709000484806],
        [-119.65906545069865, 38.38739502408541],
        [-119.60962697413615, 38.46915718247208],
        [-119.69751759913615, 38.49065815171371],
        [-119.90076466944865, 38.3012296223024],
        [-119.94470998194865, 38.3658632836034],
        [-119.97217580226115, 38.41322465329759]
    ]
])

In [ ]:
# Get Serengeti National Park
serengeti = data.filter(ee.Filter.eq('ORIG_NAME', 'Serengeti National Park')).first()
serengeti_geometry = serengeti.geometry()

# Create Map
Map = geemap.Map()
Map.centerObject(serengeti_geometry, 8)
Map.addLayer(serengeti_geometry, {'color': 'red'}, 'Serengeti Geometry')
Map

In [ ]:
# Process Serengeti data
aoi = buffer_polygon(serengeti_geometry, BUFFER, MAX_ERR)
aoi = mask_water(aoi)
modis_ic = MODIS.filter(filter_for_year(aoi, YEAR))
band_names = modis_ic.first().bandNames()
composite = modis_ic.reduce(ee.Reducer.median()).rename(band_names).clip(aoi)
image = add_indices_to_image(composite)
single_band = image.select("sur_refl_b01")
gradient = get_gradient_magnitude(single_band)

# Normalize gradient
max_val = gradient.reduce(ee.Reducer.max())
normalized_gradient = gradient.divide(max_val)

In [ ]:
# Visualization
vis_params = {
    'min': -0.5,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

Map.centerObject(aoi, 8)
Map.addLayer(single_band, vis_params, "NDVI Layer")
Map

In [ ]:
# Edge detection visualization
threshold_value = 0.1
edges = gradient.gt(threshold_value)

# Apply Gaussian blur
smoothed_gradient = gradient.focal_mean(radius=3, kernelType="circle", units="pixels")

# Visualize edges
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'white', 'red']
}

Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(gradient, vis_params)
Map

In [ ]:
# Export task
task = ee.batch.Export.image.toDrive(
    image=edges,
    description='Canny_Edges',
    folder='earth_engine_outputs',
    scale=30,
    region=aoi
)
task.start()